# AI-Powered Blog Article Generator Using LangGraph and OpenAI & Perplexity

Quickly create SEO-optimized blog articles on any topic in less than 5 minutes using LangChain, LangGraph, and OpenAI's GPT & Perplexity models. Ideal for startups, content marketers, and bloggers who need high-quality, unique content effortlessly

## Overview

- **Input Parameters:** Article Topic, Company Description  
- **Output Structure:** Structured article object with title, introduction, key headings, subheadings, body content, SEO metadata, and conclusion.  
- **Technologies:** LangChain, LangGraph, OpenAI API, Perplexity API
- **Use Cases:** Fast content generation, scalable blogging, SEO optimization automation, content marketing, and rapid publishing.

## Setup

### Jupyter Notebook

This and other tutorials are perhaps most conveniently run in a [Jupyter notebooks](https://jupyter.org/). Going through guides in an interactive environment is a great way to better understand them. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation

This tutorial requires these langchain dependencies:

In [3]:
%pip install --quiet --upgrade langchain-community langgraph langchain-core "langchain[openai]" pydantic typing_extensions

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\npden\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

In [4]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Step 1: Fetch Popular Information (Intermediate Output)

First, we need to retrieve relevant information from the internet on our topic—a collection of popular articles. To achieve this, we'll leverage Perplexity AI through the ChatPerplexity component provided by langchain_community.

The code provided assumes that your PPLX_API_KEY is set in your environment variables.
To obtain your `PPLX_API_KEY`, navigate to [https://www.perplexity.ai/settings/api](https://www.perplexity.ai/settings/api).  
Once logged in, you'll find your personal API key under the **"API Settings"** section. Copy this key and set it as an environment variable (`PPLX_API_KEY`) on your system.

In [5]:
os.environ["PPLX_API_KEY"] = getpass.getpass()

Here's a concise example demonstrating how to query Perplexity to retrieve information about popular articles related to a given topic:

In [6]:
from langchain_community.chat_models import ChatPerplexity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

perplexity_chat = ChatPerplexity(model="llama-3.1-sonar-small-128k-online")
popular_info_parser = StrOutputParser()

popular_info_prompt_template = PromptTemplate(
    template="""
You are a helpful research assistant. Find popular articles on the topic: {topic}
""",
    input_variables=["topic"]
)

fetch_popular_info_chain = popular_info_prompt_template | perplexity_chat | popular_info_parser

topic = input("Enter a topic: ")

popular_info = fetch_popular_info_chain.invoke({"topic": topic})

print(popular_info)

Here are some popular articles and sources on the topic of AI agents:

1. **World Economic Forum: "What are the risks and benefits of 'AI agents'?"**
   - This article from the World Economic Forum discusses the evolving landscape of AI agents, their benefits in enhancing productivity and efficiency, and the risks associated with their deployment, such as data privacy concerns and ethical challenges[1].

2. **AWS: "What are AI Agents?"**
   - AWS provides a comprehensive overview of AI agents, highlighting their components, how they work, and their benefits in improving productivity, reducing costs, and enhancing customer experiences. The article also discusses the challenges of using AI agents, including data privacy concerns and technical complexities[2].

3. **Boston Consulting Group (BCG): "AI Agents: What They Are and Their Business Impact"**
   - BCG's article explains the role of AI agents in automating tasks, analyzing data, and collaborating with humans. It discusses the compo

### Explanation:

- `PromptTemplate` is used to construct the instruction dynamically based on user input.
- `ChatPerplexit` executes the prompt via Perplexity AI to fetch up-to-date information.
- `StrOutputParser` converts the output from the model into a human-readable string.
- Finally, the user-provided `topic` triggers a request to Perplexity to receive popular articles.

This workflow demonstrates a simple yet effective way to integrate Perplexity AI with LangChain for research-oriented applications.

## Step 2: Implementing the Article Generation Process

Next, we need to implement the process for generating an article

The prompt we send to OpenAI's model will include:

- The information about popular articles retrieved earlier.
- Contextual meta-information about our company to ensure the generated article aligns with our company's specifics.

The following data model represents the structured format of the generated article output:


In [7]:
from typing_extensions import List
from pydantic import BaseModel, Field


class Article(BaseModel):
    title: str = Field(description="SEO-optimized title")
    introduction: str = Field(description="Introduction paragraph")
    main_points: List[str] = Field(description="Main bullet points of the article")
    conclusion: str = Field(description="Conclusion paragraph")

### Example Instance:

```json
{
  "title": "How Artificial Intelligence is Transforming the Fintech Industry",
  "introduction": "Artificial Intelligence (AI) is reshaping the financial technology sector, providing unprecedented opportunities for innovation...",
  "main_points": [
    "AI-driven fraud detection improves security",
    "Personalized customer experiences through machine learning",
    "Enhanced algorithmic trading capabilities",
    "Automation of compliance and regulatory tasks"
  ],
  "conclusion": "Integrating AI into fintech provides significant competitive advantages and operational efficiencies, positioning businesses for sustained growth."
}
```

This structured output format helps ensure clarity, consistency, and ease of integration for further processing or publication purposes.

To generate articles using OpenAI’s language models, you'll need an `OPENAI_API_KEY`. 

You can obtain this key by following these steps:

1. Navigate to [https://platform.openai.com/api-keys](https://platform.openai.com/api-keys).
2. Sign in to your OpenAI account (or register if you don't have one yet).
3. Click **"Create new secret key"**.
4. Copy the generated API key and securely store it.

After obtaining the API key, set it as an environment variable in your system:

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

Here's a concise example demonstrating how to generate an article using OpenAI and the previously defined Article data model:

In [9]:
import json

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

article_parser = JsonOutputParser(pydantic_object=Article)

article_prompt_template = PromptTemplate(
    template="""
You are an SEO copywriter. Write an article based on the information below.

Topic: {topic}
Company: {company_description}

Popular information:
{popular_info}

{format_instructions}
""",
    input_variables=["topic", "company_description", "popular_info"],
    partial_variables={"format_instructions": article_parser.get_format_instructions()},
)

generate_article_chain = article_prompt_template | llm | article_parser

company_description = input("Enter a company description: ")

article = generate_article_chain.invoke({
    "topic": topic,
    "company_description": company_description,
    "popular_info": popular_info
})

# Convert the data to a JSON formatted string with 4 spaces of indentation
json_str = json.dumps(article, indent=4)
# Print the pretty-printed JSON string
print(json_str)

{
    "title": "Unlocking Business Potential with AI Agents: A Guide by Boostcorner AI",
    "introduction": "In today's fast-paced digital landscape, the integration of AI agents into business operations is becoming increasingly essential. These intelligent systems can enhance productivity, streamline processes, and ultimately transform how companies operate. At Boostcorner AI, our team of business transformation experts and AI engineers is dedicated to helping organizations transition smoothly to AI-driven operations, ensuring they harness the full potential of these advanced technologies.",
    "main_points": [
        "AI agents are proactive systems that autonomously perform tasks to achieve specific goals, distinguishing them from reactive AI assistants.",
        "The deployment of AI agents can significantly enhance productivity and efficiency in various business sectors, as highlighted by the World Economic Forum.",
        "AWS outlines the components and functionalities of A

### Explanation:

This script demonstrates how to use OpenAI's GPT-4o-mini model to generate an SEO-optimized article in a structured JSON format using LangChain. The output follows a predefined schema based on a pydantic model (Article). 

- `ChatOpenAI`: LangChain wrapper for OpenAI's chat-based models.
- `JsonOutputParser`: Ensures the LLM output is parsed into a structured Pydantic model.

The final output is a structured SEO article tailored to the selected topic and company, ready for publishing or further processing.

## Step 3: Orchestrating the Workflow with LangGraph

To streamline the process, we will use **LangGraph** to define a workflow that:

1. Retrieves popular information related to the topic (via Perplexity).
2. Passes that information along with company metadata to OpenAI to generate a structured article.

LangGraph allows us to define this multi-step logic declaratively using a graph-based execution model.

To support a structured and type-safe workflow with LangGraph, we define the following data models:

In [10]:
from typing_extensions import TypedDict


# Input data model
class ArticleInput(TypedDict):
    topic: str
    company_description: str


# Unified workflow state
class WorkflowState(TypedDict):
    inputs: ArticleInput
    popular_info: str
    article: Article

These functions define the logic for each step in the LangGraph workflow: fetching external information and generating the article.

In [11]:
def fetch_popular_info(state: WorkflowState):
    response = fetch_popular_info_chain.invoke({"topic": state["inputs"]["topic"]})
    return {"popular_info": response}


def generate_article(state: WorkflowState):
    response = generate_article_chain.invoke({
        "topic": state["inputs"]["topic"],
        "company_description": state["inputs"]["company_description"],
        "popular_info": state["popular_info"]
    })
    return {"article": response}

We will now define the `StateGraph` that orchestrates the two main steps:

1. **Fetching popular article information** using Perplexity.  
2. **Generating a structured article** using OpenAI.

In [12]:
from langgraph.graph import END, StateGraph

# Workflow graph
workflow = StateGraph(WorkflowState)
workflow.add_node("fetch_popular_info", fetch_popular_info)
workflow.add_node("generate_article", generate_article)

workflow.set_entry_point("fetch_popular_info")
workflow.add_edge("fetch_popular_info", "generate_article")
workflow.add_edge("generate_article", END)

# Compile the app
app = workflow.compile()

Once the graph is compiled, you can invoke it with user input:

In [13]:
topic = input("Enter a topic: ")
company_description = input("Enter a company description: ")

inputs = ArticleInput(
    topic=topic,
    company_description=company_description
)

result = app.invoke({"inputs": inputs})

article = Article(**result["article"])

print("Title:", article.title)
print("\nIntroduction:", article.introduction)
print("\nMain Points:")
for point in article.main_points:
    print("-", point)
print("\nConclusion:", article.conclusion)

Title: Harnessing the Power of AI Agents: Transform Your Business with Boostcorner AI

Introduction: In today's fast-paced business landscape, the integration of AI agents is revolutionizing operations across various industries. These autonomous systems are designed to enhance productivity, streamline workflows, and empower organizations to thrive in an increasingly competitive environment. At Boostcorner AI, our team of business transformation experts and AI engineers is dedicated to helping your business transition to AI-driven operations seamlessly. This article explores the capabilities, benefits, and real-world applications of AI agents, highlighting their potential to redefine the future of work.

Main Points:
- AI agents are autonomous systems that sense and act upon their environment to achieve specific goals, significantly boosting productivity.
- By 2027, it's projected that half of the companies utilizing generative AI will have initiated AI agent pilots capable of handling 

## 🧾 Conclusion

In this tutorial, we built a structured, multi-step AI workflow for generating SEO-optimized articles using real-time web data and large language models. The workflow was implemented in three modular and composable steps:

1. **Information Retrieval via Perplexity**  
   We integrated the `ChatPerplexity` component from `langchain_community` to fetch up-to-date, relevant content on a given topic. This enables the model to work with fresh and trending data, significantly improving the relevance and quality of generated output.

2. **Structured Article Generation via OpenAI**  
   Using OpenAI's `gpt-4o-mini`, we defined a strongly typed prompt that produces content in a consistent and machine-readable format (`Article` Pydantic model). Company-specific metadata was incorporated to personalize and contextualize the final article.

3. **Workflow Orchestration with LangGraph**  
   We combined both steps into a single executable pipeline using `LangGraph`. With `StateGraph`, we clearly defined the state transitions and data flow, allowing the workflow to scale, remain testable, and be easily extended in the future (e.g., content moderation, post-processing, publishing).

By following this architecture, you can create dynamic, data-informed content generation systems that are modular, maintainable, and production-ready.